In [1]:
# NetCDF to Google Earth Engine
import os
import ee
import geemap
import geemap.colormaps as cm
import xarray as xr
from ipyleaflet import basemaps
from ipyleaflet.velocity import Velocity
geemap.ee_initialize()
import netCDF4
import scipy

# NetCDF in Remote Sensing
NetCDF is a file format that allows you to store multi-dimensional data and visualize changes over a specific domain (most often time). For instance, you can monitor temperature of an area and then visualize the change over time. Like a `.geotiff`, NetCDF stores the data in a set of matices which are flexible for both analysis and visualization and efficient when processing.

There is a very thorough [user guide](https://docs.unidata.ucar.edu/nug/current/index.html) to working with NetCDF files, although note that there is not a native interface when working with NetCDF when dealing with Python - which means that we are reliant on a series of packages to interface with the data, and not all [metadata conventions](http://cfconventions.org) are supported.

Additionally, ESRI has a good introduction to working with NetCDF in their [documentation](https://pro.arcgis.com/en/pro-app/latest/help/data/imagery/fundamentals-of-netcdf.htm) that is clearly written.

## NetCDF in Google Earth Engine
In our case, there is a primary library called xarray that acts as the primary library for reading in NetCDF files. We will use the example given with the 'wind_global.nc' NetCDF dataset that we will download from the geemap repository. We will use `xarray` to open it and then visualize it within `geemap` (although you can do this with other visualization tools, such as folium).

One key point is that you have to understand the metadata and the conventions it uses so that you can visualize and analyze the data correctly. Different     

In [2]:
file_path = os.path.abspath('wind_global.nc')
if not os.path.exists(file_path):
    url = 'https://github.com/giswqs/geemap/raw/master/examples/data/wind_global.nc'
    geemap.download_file(url)

In [3]:
ds = xr.open_dataset(file_path)
ds

<xarray.Dataset>
Dimensions:  (lat: 181, lon: 360)
Coordinates:
  * lat      (lat) float64 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
Data variables:
    u_wind   (lat, lon) float64 ...
    v_wind   (lat, lon) float64 ...
Attributes:
    centerName:      US National Weather Service - NCEP(WMC)
    disciplineName:  Meteorological products
    refTime:         2016-04-30T06:00:00.000Z

In [4]:
Map = geemap.Map(center=[44.33, -130.60], zoom = 3, interpolation='nearest')
Map.add_layer(basemaps.CartoDB.DarkMatter)

In [5]:
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data',
}
wind = Velocity(
    data=ds,
    name='Velocity',
    zonal_speed='u_wind',
    meridional_speed='v_wind',
    latitude_dimension='lat',
    longitude_dimension='lon',
    velocity_scale=0.01,
    max_velocity=20,
    display_options=display_options,
)
Map.add_layer(wind)

In [6]:
Map

Map(center=[44.33, -130.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…